In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

아래사이트를 참조하여 임베딩모델을 바꿈
https://github.com/teddylee777/langchain-kr/blob/main/07-RAG/07-RAG-1.ipynb

In [2]:
import os

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "RAG TUTORIAL"

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

웹 페이지의 내용을 로드하고, 텍스트를 청크로 나누어 인덱싱하는 과정을 거친 후, 관련된 텍스트 스니펫을 검색하여 새로운 내용을 생성하는 과정을 구현합니다.

WebBaseLoader는 지정된 웹 페이지에서 필요한 부분만을 파싱하기 위해 bs4.SoupStrainer를 사용합니다.

[참고]

bs4.SoupStrainer 는 편리하게 웹에서 원하는 요소를 가져올 수 있도록 해줍니다.
(예시)

bs4.SoupStrainer(
    "div",
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]}, # 클래스 명을 입력
)

In [ ]:
'''1.문서로드 '''
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
url = "https://n.news.naver.com/article/437/0000378416"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

In [5]:
'''2. 문서분할'''
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50)

splits = text_splitter.split_documents(docs)
len(splits)

3

In [6]:
'''3. 임베딩 & 벡터스토어 생성 '''
# 벡터스토어를 생성합니다.
# vectorstore = FAISS.from_documents(
#     documents=splits, embedding=OpenAIEmbeddings())
embeddings = HuggingFaceEmbeddings (
    model_name = "jhgan/ko-sroberta-multitask",
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings': True}
   )
vectorstore = FAISS.from_documents(
    documents = splits, embedding = embeddings
)
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.


--------------------
case: llm Chain 활용

In [19]:
from langchain import hub
''' 단계 4: 검색(Search) '''
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

''' 단계 5: 프롬프트 생성(Create Prompt) '''
prompt = hub.pull("rlm/rag-prompt")
prompt


''' 단계 6: 언어모델 생성(Create LLM) '''
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)

''' 단계 7: 체인 생성(Create Chain) '''
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
''' 단계 8: 체인 실행(Run Chain) '''
def qa_rag(question):
    response = rag_chain.invoke(question)
    print(f"[HUMAN]\n{question}\n")
    print(f"[AI]\n{response}")
    
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "부영그룹의 출산 장려 정책에 대해 설명해주세요"
print(f"문서의 수: {len(docs)}")
qa_rag(question)

문서의 수: 1
[HUMAN]
부영그룹의 출산 장려 정책에 대해 설명해주세요

[AI]
부영그룹은 출산 장려 정책으로 2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 지속할 계획이며, 연년생과 쌍둥이 자녀가 있는 경우 2억원을 받을 수 있습니다. 또한, 셋째를 낳는 경우 국민주택을 제공할 예정이며, 출산장려금을 받는 직원들의 세금 부담을 고려해 정부에 면세 제안을 할 예정입니다. 해당 정책은 사회적 분위기에서 확산되고 있습니다.


----------------------------------------
case: ConversationalRetrievalChain 활용

In [ ]:
# llm =  ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo", temperature=0)
llm =  ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
retriever = vectorstore.as_retriever(search_type='mmr', verbose=True)
rag_chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    chain_type = 'stuff',
    retriever = retriever,
    memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True, output_key='answer'),
    return_source_documents = True,
    verbose=True
    )

def qa_chat(query):
    res = rag_chain({"question":query})
    print("chat-history = ", res['chat_history'])
    return res['answer']
    
query = "부영그룹의 출산 장려 정책에 대해 설명해주세요"
print(qa_chat(query))

In [ ]:
query = "부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"
print(qa_chat(query))

In [ ]:
#이 함수는 vectorstore 객체의 delete_collection 메서드를 호출하여 컬렉션을 삭제합니다.
#이는 데이터 정리 과정에서 사용되며, 특정 데이터셋이나 정보를 저장하는 컬렉션을 제거함으로써 
#시스템의 저장 공간을 확보하고, 불필요한 데이터로 인한 혼란을 방지합니다.

# 컬렉션을 삭제합니다.
vectorstore.delete_collection()